# Assingment 12: Twitter Sentiment Classification

We work on the Kaggle challenge on [Tweet Sentiments](https://www.kaggle.com/c/tweet-sentiment-extraction/overview).

* Task: classify tweets into positive, neutral or negative sentiments.

* Method: train and apply [gensim](https://radimrehurek.com/gensim/auto_examples/index.html#documentation) [doc2vec](https://colab.research.google.com/github/keuperj/DataMining_21/blob/main/Week_10/gensim_doc2vec.ipynb) to encode the tweetes. Then train a simple [RandomForest](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html).


In [1]:
#update gensim
!pip install --upgrade gensim #numpy

In [2]:
#get the data
!git clone https://github.com/keuperj/DATA.git

Cloning into 'DATA'...
remote: Enumerating objects: 126, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 126 (delta 11), reused 39 (delta 11), pack-reused 87
Receiving objects: 100% (126/126), 185.56 MiB | 12.58 MiB/s, done.
Resolving deltas: 100% (32/32), done.
Updating files: 100% (86/86), done.


In [3]:
import pandas as pd

In [27]:
train = pd.read_csv('DATA/twitter_train.csv' , encoding = "ISO-8859-1")[:300]
test = pd.read_csv('DATA/twitter_test.csv' , encoding = "ISO-8859-1")[:300]

train['sentiment'] = train['sentiment'].replace(['neutral'], 0)
train['sentiment'] = train['sentiment'].replace(['positive'], 1)
train['sentiment'] = train['sentiment'].replace(['negative'], -1)

test['sentiment'] = test['sentiment'].replace(['neutral'], 0)
test['sentiment'] = test['sentiment'].replace(['positive'], 1)
test['sentiment'] = test['sentiment'].replace(['negative'], -1)

In [28]:
train.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",0
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,-1
2,088c60f138,my boss is bullying me...,bullying me,-1
3,9642c003ef,what interview! leave me alone,leave me alone,-1
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",-1


In [29]:
test.head()

,textID,text,sentiment
0,f87dea47db,Last session of the day http://twitpic.com/67ezh,0
1,96d74cb729,Shanghai is also really exciting (precisely -...,1
2,eee518ae67,"Recession hit Veronique Branquinho, she has to...",-1
3,01082688c6,happy bday!,1
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!,1


In [30]:
import gensim

def read_corpus(tokens_only=False):
  for i in range(len(train["text"])):
    a = str(train["text"][i]).split(" ")
    #print(a, i)
    yield gensim.models.doc2vec.TaggedDocument(a, [i])

train_corpus = list(read_corpus())
#test_corpus = list(read_corpus(tokens_only=True))

model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=40)
model.build_vocab(train_corpus)

In [31]:
print(f"Word 'boss' appeared {model.wv.get_vecattr('boss', 'count')} times in the training corpus.")

KeyError: ignored

In [32]:
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

In [51]:
ranks = []
second_ranks = []
for doc_id in range(len(train_corpus)):
    inferred_vector = model.infer_vector(train_corpus[doc_id].words)
    sims = model.dv.most_similar([inferred_vector], topn=len(model.dv))
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)

    second_ranks.append(sims[1])


import collections

counter = collections.Counter(ranks)
print(counter.items())

dict_items([(124, 2), (142, 1), (197, 1), (66, 1), (18, 2), (141, 2), (0, 66), (158, 2), (221, 1), (233, 1), (41, 5), (64, 1), (97, 1), (210, 1), (87, 2), (1, 12), (4, 7), (269, 3), (25, 3), (56, 1), (283, 1), (59, 1), (268, 1), (93, 4), (17, 1), (5, 2), (52, 1), (21, 1), (246, 2), (157, 1), (156, 1), (75, 1), (270, 1), (8, 2), (187, 1), (77, 2), (164, 1), (19, 3), (11, 1), (6, 2), (105, 1), (16, 2), (111, 1), (256, 2), (280, 1), (254, 2), (3, 6), (38, 3), (229, 1), (123, 1), (196, 1), (2, 2), (292, 1), (109, 1), (39, 2), (73, 1), (170, 1), (149, 2), (293, 1), (146, 1), (201, 2), (240, 2), (165, 2), (15, 2), (61, 1), (54, 1), (137, 1), (255, 3), (135, 2), (115, 1), (60, 1), (34, 1), (14, 2), (179, 3), (251, 2), (167, 2), (83, 2), (7, 5), (231, 1), (138, 1), (155, 1), (29, 2), (286, 1), (110, 3), (13, 2), (182, 1), (32, 1), (193, 1), (99, 1), (200, 1), (101, 1), (188, 2), (180, 1), (12, 3), (184, 2), (258, 1), (9, 1), (220, 1), (26, 3), (114, 3), (274, 2), (285, 1), (131, 1), (189, 1), 

In [50]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=30)
rf.fit(counter.values(), train["sentiment"])
#predictions_rf = rf.predict(X_test2)

TypeError: ignored